# Data Augmentation

## - Target Bracelet and Lipstick

In [83]:
import pandas as pd

df_style = pd.read_csv("Data/style/style.csv")

df_style = df_style[(df_style["product_name"]== "lipstick") | (df_style["product_name"]== "bracelet")]
df_style.to_csv("Data/style_DataAugmentation/style_DA.csv")
imageNameList = df_style["file"].values.tolist()

## - Save them in other File

In [14]:
from PIL import Image

root_dir = 'Data/style/'

for i in range(len(imageNameList)):
    img = Image.open(root_dir+imageNameList[i])
    img.save("Data/style_DataAugmentation/"+imageNameList[i])

## - Data Transformation

In [89]:
import torch
import torchvision.transforms as transforms
from torchvision.utils import save_image
from CnnAlgoClass.LoadData import LoadData
from torch.utils.data import DataLoader
from CnnAlgoClass.AlexNet import AlexNet
from CnnAlgoClass.Utils import imshow, modelTrain, modelAccurcy, saveModel, loadModel

In [53]:
tf = transforms.Compose([
    transforms.ToTensor(),
    transforms.ToPILImage(),
    transforms.Resize((170, 170)),
    transforms.RandomCrop((150, 140)),
    transforms.ColorJitter(brightness=0.5),
    transforms.RandomRotation(20),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomGrayscale(0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.0, 0.0, 0.0], std=[1.0, 1.0, 1.0])
])

In [54]:
dataset = LoadData(csv_file='Data/style_DataAugmentation/style_AG.csv', root_dir='Data/DataAugmentation_style', transform = tf)

In [55]:
i = j = 0
root_dir = 'Data/style_DataAugmentation/'

for img, label in dataset:
    if int(label) == 1:
        name = "1_3_"+str(i)+'.png'
        i+=1
    if int(label) == 7:
        name = "7_3_"+str(j)+'.png'
        j+=1
    save_image(img, root_dir+name)

# Training / Testing

In [88]:
tf = transforms.Compose([
  transforms.Resize(256),
  transforms.CenterCrop(224),
  transforms.ToTensor(),
  transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

dataset = LoadData(csv_file='Data\style_DataAugmentation\style_1.csv', root_dir='Data\style_DataAugmentation', transform = tf)

data_loader = DataLoader(dataset=dataset, batch_size=20, shuffle=True)

classes = ('shoes', 'lipstick', 'handbag', 'nail polish', 'necklace', 'watches', 'ring', 'bracelet', 'boots', 'earrings')

In [90]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
alex = AlexNet().to(device)

loadModel(alex, './models/AlexNet_Adam_VF.pth')

optimizer = torch.optim.Adam(alex.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

## - Training 1

In [91]:
modelTrain(alex, data_loader, optimizer, criterion, epoch=1)

--- Training started ---
epoch: 1 loss: 0.8553
--- Finished Training ---


## - Testing 1

In [92]:
dataset1 = LoadData(csv_file='Data\style\style.csv', root_dir='Data\style', transform = tf)
data_loader1 = DataLoader(dataset=dataset1, batch_size=20, shuffle=True)

modelAccurcy(alex, data_loader1, classes)

----- Accuracy for each class ----
- shoes : 78.77 %
- lipstick : 89.04 %
- handbag : 81.15 %
- nail polish : 28.57 %
- necklace : 56.00 %
- watches : 73.72 %
-  ring : 41.84 %
- bracelet : 81.82 %
- boots : 62.66 %
- earrings : 67.07 %
--------------------------------------------------
Accuracy of the network on the test images: 69.69 %


## - Training 2

In [93]:
modelTrain(alex, data_loader, optimizer, criterion, epoch=3)

--- Training started ---
epoch: 1 loss: 0.7550
--- Finished Training ---


## - Testing 2

In [94]:
modelAccurcy(alex, data_loader1, classes)

----- Accuracy for each class ----
- shoes : 83.15 %
- lipstick : 93.42 %
- handbag : 66.59 %
- nail polish : 50.65 %
- necklace : 60.67 %
- watches : 71.53 %
-  ring : 71.94 %
- bracelet : 67.77 %
- boots : 77.22 %
- earrings : 75.00 %
--------------------------------------------------
Accuracy of the network on the test images: 73.58 %


In [95]:
saveModel(alex, './models/AlexNet_Adam_DA.pth')

'Model saved successfully'

## - Training 3

In [96]:
modelTrain(alex, data_loader, optimizer, criterion, epoch=1)

--- Training started ---
epoch: 1 loss: 0.7492
--- Finished Training ---


## - Testing 3

In [97]:
modelAccurcy(alex, data_loader1, classes)

----- Accuracy for each class ----
- shoes : 89.93 %
- lipstick : 93.42 %
- handbag : 82.58 %
- nail polish : 22.08 %
- necklace : 65.33 %
- watches : 60.58 %
-  ring : 41.33 %
- bracelet : 74.38 %
- boots : 63.29 %
- earrings : 82.32 %
--------------------------------------------------
Accuracy of the network on the test images: 72.85 %
